In [1]:
import cv2
import os

In [2]:
MIN_CONTOUR_AREA = 80                       # minimum area of valid contour
MAX_CONTOUR_AREA = 1000                      # maximum area of valid contour

RESIZED_IMAGE_WIDTH = 20                    # width of resized image
RESIZED_IMAGE_HEIGHT = 30                   # height of resized image

# folder with training images
TRAINING_IMAGES_PATH = os.getcwd() + '/dataset/'
# training alphabet image
TRAINING_ALPHA = os.getcwd() + '/train_images/' + "train_alpha1.png"
# training digit image
TRAINING_DIGIT = os.getcwd() + '/train_images/' + "train_digit1.png"

In [3]:
class ContourWithData():
    npaContour = None           # contour
    boundingRect = None         # bounding rect for contour
    intRectX = 0                # bounding rect top left corner x location
    intRectY = 0                # bounding rect top left corner y location
    intRectWidth = 0            # bounding rect width
    intRectHeight = 0           # bounding rect height
    fltArea = 0.0               # area of contour

    # calculate bounding rect info
    def calculateRectTopLeftPointAndWidthAndHeight(self):
        [intX, intY, intWidth, intHeight] = self.boundingRect
        self.intRectX = intX
        self.intRectY = intY
        self.intRectWidth = intWidth
        self.intRectHeight = intHeight

    # check if coontour is a valid character
    def checkIfContourIsValid(self):
        if self.fltArea < MIN_CONTOUR_AREA or self.fltArea > MAX_CONTOUR_AREA:
            return False
        return True

In [4]:
# read in training alpha image
imgTrainingAlpha = cv2.imread(TRAINING_ALPHA)
# read in training digit image
imgTrainingDigit = cv2.imread(TRAINING_DIGIT)

if imgTrainingAlpha is None:                                # if image was not read successfully
    print("error: alpha image not read from file \n\n")
    exit(0)
# end if

if imgTrainingDigit is None:                                # if image was not read successfully
    print("error: digit image not read from file \n\n")
    exit(0)
# end if

In [5]:
# get grayscale image
imgGrayAlpha = cv2.cvtColor(imgTrainingAlpha, cv2.COLOR_BGR2GRAY)

# get threshold image
imgThreshAlpha = cv2.adaptiveThreshold(
    imgGrayAlpha, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)

In [6]:
# get grayscale image
imgGrayDigit = cv2.cvtColor(imgTrainingDigit, cv2.COLOR_BGR2GRAY)

# get threshold image
imgThreshDigit = cv2.threshold(
    imgGrayDigit, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

In [7]:
npaContoursAlpha, _ = cv2.findContours(
    imgThreshAlpha.copy(),          # input image, use a copy since the function will modify this image in the course of finding contours
    cv2.RETR_EXTERNAL,              # retrieve the outermost contours only
    cv2.CHAIN_APPROX_SIMPLE)        # compress horizontal, vertical, and diagonal segments and leave only their end points

In [8]:
npaContoursDigit, _ = cv2.findContours(
    imgThreshDigit.copy(),          # input image, make sure to use a copy since the function will modify this image in the course of finding contours
    cv2.RETR_EXTERNAL,              # retrieve the outermost contours only
    cv2.CHAIN_APPROX_SIMPLE)        # compress horizontal, vertical, and diagonal segments and leave only their end points

In [9]:
AlphaContours = []
DigitContours = []

In [10]:
for npaContour in npaContoursAlpha:                 # for each contour
    # instantiate a contour with data object
    contourWithData = ContourWithData()
    # assign contour to contour with data
    contourWithData.npaContour = npaContour
    # get the bounding rect
    contourWithData.boundingRect = cv2.boundingRect(
        contourWithData.npaContour)
    # get bounding rect info
    contourWithData.calculateRectTopLeftPointAndWidthAndHeight()
    # calculate the contour area
    contourWithData.fltArea = cv2.contourArea(contourWithData.npaContour)
    # add contour with data object to list of all contours with data
    if contourWithData.checkIfContourIsValid():
        AlphaContours.append(contourWithData)

# sort contours from left to right
AlphaContours.sort(key=lambda x: x.intRectX)

In [11]:
char = 65
for i in range(len(AlphaContours)):                 # for each contour
    npaContour = AlphaContours[i]
    # increment character if new column
    if npaContour.intRectX - AlphaContours[i-1].intRectX >= AlphaContours[i-1].intRectWidth:
        char += 1
    # crop char out of threshold image
    imgROI = imgThreshAlpha[npaContour.intRectY: npaContour.intRectY + npaContour.intRectHeight,
                            npaContour.intRectX: npaContour.intRectX + npaContour.intRectWidth]
    # resize image, this will be more consistent for recognition and storage
    imgROIResized = cv2.resize(
        imgROI, (RESIZED_IMAGE_WIDTH, RESIZED_IMAGE_HEIGHT))

    # write image to file
    dir = TRAINING_IMAGES_PATH + chr(char) + '/'
    try:
        os.chdir(dir)
    except:
        os.mkdir(dir)
    cv2.imwrite(dir + str(len(os.listdir(dir))) + ".png", imgROIResized)
    print(chr(char))
# end for

A
A
A
A
A
B
B
B
B
B
C
C
C
C
C
D
D
D
D
D
E
E
E
E
E
F
F
F
F
F
G
G
G
G
G
H
H
H
H
H
I
I
I
I
I
J
J
J
J
J
K
K
K
K
K
L
L
L
L
L
M
M
M
M
M
N
N
N
N
N
O
O
O
O
O
P
P
P
P
P
Q
Q
Q
Q
Q
R
R
R
R
R
S
S
S
S
S
T
T
T
T
T
U
U
U
U
U
V
V
V
V
V
W
W
W
W
W
X
X
X
X
X
Y
Y
Y
Y
Y
Z
Z
Z
Z
Z


In [12]:
for npaContour in npaContoursDigit:                 # for each contour
    # instantiate a contour with data object
    contourWithData = ContourWithData()
    # assign contour to contour with data
    contourWithData.npaContour = npaContour
    # get the bounding rect
    contourWithData.boundingRect = cv2.boundingRect(
        contourWithData.npaContour)
    # get bounding rect info
    contourWithData.calculateRectTopLeftPointAndWidthAndHeight()
    # calculate the contour area
    contourWithData.fltArea = cv2.contourArea(contourWithData.npaContour)
    # add contour with data object to list of all contours with data
    if contourWithData.checkIfContourIsValid():
        DigitContours.append(contourWithData)

# sort contours from left to right
DigitContours.sort(key=lambda x: x.intRectX)

In [13]:
char = 48
for i in range(len(DigitContours)):                      # for each contour
    npaContour = DigitContours[i]
    # increment character if new column
    if npaContour.intRectX - DigitContours[i-1].intRectX >= DigitContours[i-1].intRectWidth:
        char += 1
        # cv2.waitKey()
    # crop char out of threshold image
    imgROI = imgThreshDigit[npaContour.intRectY: npaContour.intRectY + npaContour.intRectHeight,
                            npaContour.intRectX: npaContour.intRectX + npaContour.intRectWidth]
    # resize image, this will be more consistent for recognition and storage
    imgROIResized = cv2.resize(
        imgROI, (RESIZED_IMAGE_WIDTH, RESIZED_IMAGE_HEIGHT))
    # write image to file
    dir = TRAINING_IMAGES_PATH + chr(char) + '/'
    try:
        os.chdir(dir)
    except:
        os.mkdir(dir)
    cv2.imwrite(dir + str(len(os.listdir(dir))) + ".png", imgROIResized)
    print(chr(char))
# end for

0
0
0
0
0
1
1
1
1
1
2
2
2
2
2
3
3
3
3
3
4
4
4
4
4
5
5
5
5
5
6
6
6
6
6
7
7
7
7
7
8
8
8
8
8
9
9
9
9
9
